### Steps for this exercise
##### 1. necessary imports 
##### 2. setting the model parameter and intializing either openai or ollama
##### 3. Writing the Website Utility Class to scrape data even links this time
##### 4. get all the necessary links for the given company's homePage link using the get_all_links(url) function which communicates with ollama using the system and user prompts
##### 5. Then we create the brochure and additionaly stream if the client supports after communication with the ollama api using the system and user prompt.

In [43]:
import requests
import ollama
import json
from bs4 import BeautifulSoup
from openai import OpenAI
from IPython.display import Markdown, display, update_display
import re

In [44]:
MODEL = 'gemma3:4b'
ollama_via_openai = OpenAI(base_url="https://localhost:11434/v1",api_key="ollama")

In [45]:
## A utility class for scraping Websites now also with links

class Website:
    def __init__(self,url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body,'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator='\n',strip=True)
        else:
            self.text=""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents: \n{self.text}\n"

In [46]:
sec = Website("https://www.satyasai.co.in")
sec.links

['https://satyasai.co.in/contact-us',
 'https://satyasai.co.in',
 '#search',
 '#',
 'javascript:void(0)',
 '#',
 '#',
 '#',
 '#',
 '#',
 '#',
 'https://satyasai.co.in/',
 'https://satyasai.co.in/about-us/',
 'https://satyasai.co.in/services/',
 'https://satyasai.co.in/projects/',
 'https://satyasai.co.in/contact-us/',
 '#',
 '#',
 'javascript:;',
 'javascript:;',
 'javascript:;',
 'javascript:;',
 'javascript:;',
 'https://www.google.com',
 'https://rss.com',
 'https://www.facebook.com',
 'https://twitter.com',
 'https://in.linkedin.com',
 'https://satyasai.co.in/about-us/',
 'https://satyasai.co.in/services/',
 'https://satyasai.co.in/projects/',
 'https://satyasai.co.in/contact-us/']

In [47]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON object format as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [48]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms o f Service, Privacy, email links \n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [49]:
def get_links(url):
    website = Website(url)
    response = ollama.chat(
        model=MODEL,
        messages=[
            {"role":"system","content":link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
        ]
    )
    result = response["message"]["content"]

    cleaned_result = re.sub(r"^```(?:json)?\n|\n```$", "", result.strip())

    return json.loads(cleaned_result)

In [50]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/'},
  {'type': 'models', 'url': 'https://huggingface.co/models/'},
  {'type': 'datasets', 'url': 'https://huggingface.co/datasets/'},
  {'type': 'spaces', 'url': 'https://huggingface.co/spaces/'},
  {'type': 'documentation', 'url': 'https://huggingface.co/docs/'},
  {'type': 'enterprise', 'url': 'https://huggingface.co/enterprise/'},
  {'type': 'pricing', 'url': 'https://huggingface.co/pricing/'},
  {'type': 'documentation', 'url': 'https://huggingface.co/docs/'},
  {'type': 'documentation', 'url': 'https://huggingface.co/docs/'},
  {'type': 'documentation', 'url': 'https://huggingface.co/docs/'},
  {'type': 'documentation', 'url': 'https://huggingface.co/docs/'},
  {'type': 'documentation', 'url': 'https://huggingface.co/docs/'}]}

#### Now we have all the links in the given website/companyLink. We will now move on to the next step of actually creating the Brochure.

In [51]:
def get_all_details(url):
    result = "Landing Page"
    result += Website(url).get_contents()
    links = get_links(url)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [52]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [53]:
def get_brochure_user_prompt(company_name,url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5000] # Truncate if more than 5000 characters
    return user_prompt

In [63]:
def create_brochure(companyName,url):
    response = ollama.chat(
        model=MODEL,
        messages=[
            {"role":"system","content":system_prompt},
            {"role":"user","content":get_brochure_user_prompt(companyName,url)}
        ],
    )   
    display(Markdown(response["message"]["content"])) 

In [64]:
create_brochure("satyasai","https://www.satyasai.co.in")

Okay, here’s a brochure designed for Satyasai Construction, incorporating the information from the provided web pages.

---

**Satyasai Construction: Delivering Seamless Material Solutions**

**(Brochure Cover Image: A dynamic image of materials handling equipment in operation – ideally a conveyor system or forklift.)**

---

**For Prospective Customers**

**Headline:**  Streamline Your Construction Project with Expert Materials Handling

**Introduction:** Satyasai Engineering & Construction is a leading provider of innovative materials handling solutions for the construction industry. With 15+ years of experience and a commitment to quality and efficiency, we help you optimize your project’s workflow, reduce downtime, and ensure the safe and timely delivery of materials.

**Key Services:**

*   **Facility Design & Construction:**  Customized facilities designed for optimal workflow and efficiency.
*   **Conveyor Systems & Automation:** Enhance material flow with cutting-edge conveyor systems and automation solutions.
*   **Equipment Installation & Maintenance:**  Reliable installation and ongoing maintenance for long-term equipment performance.
*   **System Upgrades & Retrofitting:** Modernize your systems for improved performance and reduced costs.
*   **Waste Removal & Recycling:**  Sustainable solutions for responsible waste management on-site.

**(Image:  A small graphic representing the core services – perhaps icons for each.)**

---

**For Investors**

**Headline:**  Investing in a Growing Industry Leader

**About Satyasai:**  Satyasai Engineering & Construction is rapidly gaining recognition for its specialized expertise in materials handling construction.  We operate with a proven track record of successful project delivery and a dedicated team focused on innovation and client satisfaction. 

**Key Highlights:**

*   **Strong Market Position:**  A key player in the materials handling sector, with established operations across key refinery and industrial sites in India.
*   **Experienced Team:**  A workforce of 450+ active experts, demonstrating commitment and skills.
*   **Client Portfolio:**  Working with major Indian Oil Corporation Limited, Engineers India Limited, and National Aluminium Corporation Limited – showcasing trust and reliability.
*   **Growth Potential:**  The demand for efficient materials handling solutions is steadily increasing, providing a solid foundation for future growth. 
*   **Financial Stability:** (Note:  This information is not available from the given pages, but would be crucial for investors.)

**(Image: A graph showcasing growth trends in the materials handling sector.)**

---

**For Recruits/Potential Employees**

**Headline:**  Join a Team Dedicated to Innovation and Excellence

**About Satyasai Culture:** At Satyasai, we’re more than just a construction company; we’re a team committed to delivering exceptional results while fostering a collaborative and supportive environment.

**What We Offer:**

*   **Innovative Work:** Be part of a team that's constantly seeking new and improved solutions for materials handling challenges.
*   **Growth Opportunities:**  We invest in our employees' development through training and advancement opportunities.
*   **Teamwork & Collaboration:**  Work alongside experienced professionals in a positive and supportive atmosphere.
*   **Commitment to Safety:**  Safety is our top priority – we maintain rigorous standards to ensure the well-being of our team.
*  **Positive Work Environment:** Our aim is to ensure you have a friendly service provider and a good management team to lead your growth. 

**Career Paths:** We have a diverse range of roles available, including:

*   Project Engineers
*   Equipment Technicians
*   Operations Specialists
*   Maintenance Personnel

**Contact Us:**  We are always looking for talented individuals to join our team. Please send your resume and cover letter to admin@satyasai.co.in or call +91 99384 73499 / +91 90400 73496.

**(Image: A photo of a diverse team working together.)**



---

**(Back Cover)**

**Satyasai Engineering & Construction**
H.O- Chunabellari, PO: Paradeepgarh, Dist.Jagatsinghapur, Odisha, 754141
Corporate Office: Haldia Refinery, PO: Haldia Refinery Campus, PS: Haldia, West Bengal, 721602
admin@satyasai.co.in
+91 99384 73499 / +91 90400 73496

**(Website Address:  www.satyasai.co.in (Placeholder - Actual Website Needs to be Added)**

---

**Note:**  This brochure is based on the information provided.  To make it truly compelling, it would benefit from higher-quality images, more detailed service descriptions, and specific case study examples.  Also, accurate financial data would be essential for the investor brochure.

In [ ]:


from IPython.display import display, Markdown

def create_stream_brochure(company_name, url):
    stream = ollama.chat(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
        stream=True  # Only if your Ollama client supports this
    )

    response = ""
    display_handle = display(Markdown(""), display_id=True)

    for chunk in stream:
        # Adjust this depending on Ollama's streaming format
        delta = chunk.get("message", {}).get("content", "")
        response += delta or ''
        response = response.replace("```", "").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [55]:
create_stream_brochure("satyasai","https://www.satyasai.co.in")

Okay, here's a brochure for Satyasai Engineering & Construction, built from the information you provided. It's designed to appeal to prospective customers, investors, and recruits.

---

## **Satyasai Engineering & Construction: Moving Your Project Forward**

**(Image: A dynamic image of a conveyor belt moving materials, perhaps with construction equipment in the background – ideally a professional, high-quality photo)**

**A Trusted Partner in Materials Handling Construction**

**For Customers:**

**Satyasai provides comprehensive materials handling solutions, ensuring your construction projects run smoothly and efficiently.**

*   **Our Expertise:** With 15+ years of experience, we specialize in the design, construction, and maintenance of materials handling systems, including:
    *   Facility Design & Construction
    *   Conveyor Systems & Automation
    *   Equipment Installation & Maintenance
    *   System Upgrades & Retrofitting
*   **We Serve a Wide Range of Industries:** Including Indian Oil Corporation, Engineers India Limited, National Aluminium Corporation Limited, and more.
*   **Key Benefits:**
    *   **Increased Efficiency:** Optimized workflows and faster project completion.
    *   **Reduced Costs:**  Streamlined material movement minimizes delays and waste.
    *   **Reliable Support:** Experienced team and ongoing maintenance services.
*   **Client Testimonials:** “Satyasai Engineering & Construction Has Solid Strength with Sufficient Handling Equipment, Tools and Tackles and Well Proffessional Manpower.” – Indian Oil Corporation Limited

**(Image: A collage showcasing various equipment - a conveyor belt, a forklift, a small automated system – demonstrating the breadth of capabilities.)**


**For Investors:**

**Satyasai Engineering & Construction: A Growing Opportunity**

*   **Strong Market Position:**  A leader in materials handling construction in India, with a proven track record of successful projects.
*   **Significant Growth Potential:**  The construction industry in India is booming, creating substantial demand for our services.
*   **Key Metrics:**
    *   **Active Experts:** 450
    *   **Satisfied Clients:** 670
    *   **Project Complete:** 1500+
*   **Focus Areas:** We’re committed to innovative solutions and expansion within key sectors.
* **Reliable Team:** Our experienced management team and skilled workforce ensure efficient operations and client satisfaction.

**For Recruits:**

**Join the Satyasai Team: Build Your Career in a Growing Industry!**

*   **Opportunity:**  Satyasai is rapidly expanding, offering exciting career opportunities for talented individuals.
*   **We Offer:**
    *   **Training & Development:** Invested in our employee's skills.
    *   **Diverse Roles:** Opportunities in engineering, project management, maintenance, and more.
    *   **Dynamic Work Environment:**  A culture of teamwork, innovation, and commitment to quality.
*   **Skills We Value:**  Mechanical Engineering, Electrical Engineering, Automation, Project Management.

**(Image: A team photo of employees looking professional and engaged.)**


**Contact Us:**

*   **Head Office:** Chunabellari, PO: Paradeepgarh, Dist.Jagatsinghapur, Odisha, 754141
*   **Corporate Office:** Haldia Refinery, PO: Haldia Refinery Campus, PS: Haldia, West Bengal, 721602
*   **Email:** admin@satyasai.co.in
*   **Phone:** +91 99384 73499 / +91 90400 73496
*   **Website:** [Replace with actual website address when available]
*   **Social Media Links (if available)**

---

**Note:** I've tried to create a brochure that is visually appealing and informative, incorporating the key elements from the website.  I've added placeholders like "[Replace with actual website address when available]" to ensure you can populate the final brochure with accurate details.  The image suggestions are important – high-quality images will significantly enhance the brochure’s impact.